# Titanic 
## A machine learning introduction

![The Titanic](https://de.wikipedia.org/wiki/RMS_Titanic#/media/Datei:Cd51-1000g.gif)

# Settings

In [52]:
# settings
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
#To revert to the original setting :
# InteractiveShell.ast_node_interactivity = "last_expr"

# Imports

In [53]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling # dataframe profiling @
import os

In [54]:
# walk file structure and find input data
input_files=[]
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        path=os.path.join(dirname, filename)
        if path.endswith('submission.csv'):
            path_gs=path
        elif path.endswith('test.csv'):
            path_test=path
        elif path.endswith('train.csv'):
            path_train=path
print(path_gs,path_test,path_train)            

kaggle/input\gender_submission.csv kaggle/input\test.csv kaggle/input\train.csv


# Data Frame Creation

In [55]:
# create a dataframe for each csv file
df_gs=pd.read_csv(path_gs)
df_test=pd.read_csv(path_test)
df_train=pd.read_csv(path_train)

# display
# df_gs.head()
# df_test.head()
# df_train.head()

# Wrangling

In [56]:
# # df_test change string variable to int
# df=df_test
# df.loc[df_test.Sex=='female','gender']=0
# df.loc[df_test.Sex=='male','gender']=1
# df.loc[df_test.Cabin.isna(),'Cabn']=0
# df.loc[df_test.Cabin.notnull(),'Cabn']=1
# df.loc[df_test.Embarked=='Q','Embark']=0
# df.loc[df_test.Embarked=='S','Embark']=1
# df.loc[df_test.Embarked=='C','Embark']=2
# #df.head()
# df_test.drop(['Sex','Cabin','Name','Embarked'],axis=1,inplace=True)
# df.head()

# Profile reports

In [57]:
#profile report fpor each df
# source : https://pypi.org/project/pandas-profiling/
#df_test.profile_report(title='Titanic test')

In [58]:
#profile report for notebook export
#source : https://pypi.org/project/pandas-profiling/

#gender sub
profile_gs = df_gs.profile_report(title='Titanic gender submission')
profile_gs.to_file(output_file="kaggle/profile_report/Titanic gender submission.html")

#gender sub
profile_test = df_test.profile_report(title='Titanic test')
profile_test.to_file(output_file="kaggle/profile_report/Titanic test.html")

#gender sub
profile_train = df_train.profile_report(title='Titanic train')
profile_train.to_file(output_file="kaggle/profile_report/Titanic train.html")